# Load environment variables.

## These are used to configure the mlflow library. And should point to your mlflow instance.  
### MLFLOW_TRACKING_URI
this is the main mlflow server, same as where the webui runs.  
Normally on port 5000
### MLFLOW_S3_ENDPOINT_URL
this is the S3/artifact store. Is a minio instance(see compose file).  
It is required because the mlflow client will push artifacts directly to the s3 store, not the main mlflow api.  
Default runs on port 9000


This example uses jupyter magicks to import the .env file.  
Be careful, the .env for the docker-compose and the .env for the project are treated separately, and this should be the .env from the project root.  

**Magicks for notebooks:** 

In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

**Or dotenv for normal python**

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

**Required env variables**  
These environment variables need to be set in order for mlflow to work.  
You might be able to live without the s3 and aws variables, but then log_artifact and log_model will probably not work.  
The URI can be set to 127.0.0.1/localhost if training on the same machine as mlflow host.


In [3]:
import os
print("MLFLOW_TRACKING_URI:", os.environ.get("MLFLOW_TRACKING_URI"))
print("MLFLOW_S3_ENDPOINT_URL:", os.environ.get("MLFLOW_S3_ENDPOINT_URL"))
print("AWS_ACCESS_KEY_ID:", os.environ.get("AWS_ACCESS_KEY_ID"))
print("AWS_SECRET_ACCESS_KEY:", os.environ.get("AWS_SECRET_ACCESS_KEY"))#%%


MLFLOW_TRACKING_URI: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000
MLFLOW_S3_ENDPOINT_URL: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:9000
AWS_ACCESS_KEY_ID: minio
AWS_SECRET_ACCESS_KEY: minio123


**Generic MLFLow Setup**
Import it, then check the URI, if it matches, import and env variables are set. 

In [4]:
import mlflow
print(mlflow.get_tracking_uri())

http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000


# Setting up an experiment
Experiments are like directories, and can be used to group different runs together.  
For example:  
- a evaluation over different models with the same usecase.
- different runs during prototyping of a model (Such as ARIMA)

This can be done with the code below.  
If the experiment does not exists, a new experiment will be created (so dont add for loops or cv params to the experiment name).  
If you dont set an experiment, runs may just be added to the default experiment(if available)


In [5]:
mlflow.set_experiment(experiment_name="MLFlow_Showcase")

<Experiment: artifact_location='s3://mlflow/1', creation_time=1764506916586, experiment_id='1', last_update_time=1764506916586, lifecycle_stage='active', name='MLFlow_Showcase', tags={}>

# Logging a run
Runs are instances of a model training. 
For example, training a model is a single run, It does not matter how many epochs, as long as its just a single model to be trained, its a run.  
If the model run is aborted or fails, and is then restarted, it will be a new run.  
If the same playbook is executed after the run is completed, it will be a new run. 
Important: If the run is not stopped(visible in mlflow ui), there may be an error "run is already active). Use try catch, or alternatively the "with" statement. 

If we do a gridsearch over different hyper parameters, each of them would be a new run. Thats what nested runs are for, we will see them later. 


In [6]:
import mlflow
mlflow.set_experiment(experiment_name="MLFlow_ExperimentWithRuns") # setup experiment

with mlflow.start_run(run_name="SimpleRun"):
    mlflow.log_param(key="UselessParam", value="ThisValueIsUseles")
    # model training code


🏃 View run SimpleRun at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/193620decc0f437e93515cc0639d327c
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2


# Logging parameters, artifacts, figures, metrics.
Logging in mlflow is done using the log_param, log_metrics, log_artifacts functions. See the training code below for samples.  
check the docs for more log_XYZ functions, there are a lot of them, for example figures(matplotlib) can be logged as well. Or files.


In [7]:
# These need to be used when a run is already started using start_run
mlflow.log_param(key="random_state", value= "42")
mlflow.log_metric("f1_score", "00324")


# Simple SK Learn Example 
Including setup, loggin params, logging metrics.  
The model is also logged, it does make sense to not rely on mlflow for model storage though.  

In [8]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# this is only required because there may be already a run from above. This should not happen during "normal" coding
mlflow.end_run()

# Set experiment
mlflow.set_experiment(experiment_name="MLFlow_ExperimentWithRuns")

# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# Start MLflow run
with mlflow.start_run(run_name="RandomForestClassifier"):
    
    # Define hyperparameters
    n_estimators = 10
    max_depth = 3
    random_state = 42
    
    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)
    
    # Train model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state
    )
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "random_forest_model")
    
    print(f"Model trained successfully!")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")


🏃 View run beautiful-snail-350 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/8fc8eb6fc9e243febaf5b986245760f3
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2


2025/11/30 13:51:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 13:51:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/30 13:51:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model trained successfully!
Accuracy: 1.0000
F1 Score: 1.0000
🏃 View run RandomForestClassifier at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/f3e81e2244094b1e932b3a3d799ddcdd
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2


# Using nested runs. 

I use this mostly during grid search.  

## The problem when not using them:
Because every single time a new parameter is used, its technically a new model.  
This leads us to having N runs, one for each parameter tested.  
Will fill up storage quite quickly and will make it hard to filter for runs in the ui. 


## solution:  
Nested runs let you organize all those parameter combinations under a single parent run.  
Instead of having 50 separate runs cluttering your experiment, you get 1 parent run with 50 child runs inside it.  

## How it works
The parent run tracks the overall grid search.  
Each child run tracks a specific parameter combination.  
You mark child runs with nested=True when starting them.  

## How it will look
Storage is cleaner because runs are grouped hierarchically.  
The UI shows an expandable tree structure instead of a flat list.  
You can compare all variations in one place without filtering through unrelated runs.  
The parent run stores summary info while children store detailed metrics.  

## When to use
Grid search and hyperparameter tuning is the obvious one.  
Cross-validation where each fold is a child run.  
Multi-stage pipelines where each stage gets its own nested run.  
Ensemble methods with a parent for the ensemble and children for individual models.  


# Notes about the cell below

In [9]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import itertools

# Set experiment
mlflow.set_experiment(experiment_name="MLFlow_ExperimentWithRuns")

# Load dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# Define parameter grid
param_grid = {
    'n_estimators': [5, 10, 20],
    'max_depth': [2, 3, 5],
    'min_samples_split': [2, 5]
}

# Generate all combinations
param_combinations = [dict(zip(param_grid.keys(), v)) 
                      for v in itertools.product(*param_grid.values())]

print(f"Running grid search with {len(param_combinations)} parameter combinations...\n")

# Parent run for the entire grid search
with mlflow.start_run(run_name="GridSearch_ParentRun") as parent_run:
    
    # Log grid search configuration
    mlflow.log_param("total_combinations", len(param_combinations))
    mlflow.log_param("param_grid", str(param_grid))
    
    best_accuracy = 0
    best_params = None
    all_results = []
    
    # Iterate through parameter combinations
    for idx, params in enumerate(param_combinations, 1):
        
        # Nested child run for each parameter combination
        with mlflow.start_run(run_name=f"Run_{idx}", nested=True):
            
            print(f"[{idx}/{len(param_combinations)}] Training with params: {params}")
            
            # Log parameters
            for param_name, param_value in params.items():
                mlflow.log_param(param_name, param_value)
            
            # Train model
            model = RandomForestClassifier(
                n_estimators=params['n_estimators'],
                max_depth=params['max_depth'],
                min_samples_split=params['min_samples_split'],
                random_state=42
            )
            model.fit(X_train, y_train)
            
            # Make predictions
            y_pred = model.predict(X_test)
            
            # Calculate metrics
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            
            # Log metrics
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            
            # Log model
            mlflow.sklearn.log_model(model, "model")
            
            # Track results
            all_results.append({
                'params': params,
                'accuracy': accuracy,
                'f1_score': f1
            })
            
            # Update best model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params
            
            print(f"  → Accuracy: {accuracy:.4f}, F1: {f1:.4f}\n")
    
    # Log best results to parent run
    mlflow.log_metric("best_accuracy", best_accuracy)
    mlflow.log_param("best_params", str(best_params))
    
    # Train and log best model
    print(f"Best parameters: {best_params}")
    print(f"Best accuracy: {best_accuracy:.4f}\n")
    
    best_model = RandomForestClassifier(**best_params, random_state=42)
    best_model.fit(X_train, y_train)
    mlflow.sklearn.log_model(best_model, "best_model")
    
    # Create and log summary artifact
    summary = "Grid Search Results\n" + "="*50 + "\n\n"
    summary += f"Total combinations tested: {len(param_combinations)}\n"
    summary += f"Best accuracy: {best_accuracy:.4f}\n"
    summary += f"Best parameters: {best_params}\n\n"
    summary += "All Results:\n" + "-"*50 + "\n"
    
    for i, result in enumerate(sorted(all_results, key=lambda x: x['accuracy'], reverse=True), 1):
        summary += f"{i}. Accuracy: {result['accuracy']:.4f} | Params: {result['params']}\n"
    
    with open("grid_search_summary.txt", "w") as f:
        f.write(summary)
    
    mlflow.log_artifact("grid_search_summary.txt")   # this is one of the possibilities to log files. 
    
    print("="*50)
    print("Grid search complete!")
    print("="*50)

Running grid search with 18 parameter combinations...

[1/18] Training with params: {'n_estimators': 5, 'max_depth': 2, 'min_samples_split': 2}


2025/11/30 13:51:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 13:51:50 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/30 13:51:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  → Accuracy: 1.0000, F1: 1.0000

🏃 View run Run_1 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/f0272704bc404d55a0597e1b20c51185
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2
[2/18] Training with params: {'n_estimators': 5, 'max_depth': 2, 'min_samples_split': 5}


2025/11/30 13:51:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 13:51:52 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/30 13:51:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  → Accuracy: 1.0000, F1: 1.0000

🏃 View run Run_2 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/80e3add7572e4c7398f327b715bcd711
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2
[3/18] Training with params: {'n_estimators': 5, 'max_depth': 3, 'min_samples_split': 2}


...

2025/11/30 13:52:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 13:52:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/30 13:52:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  → Accuracy: 1.0000, F1: 1.0000

🏃 View run Run_10 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/872f038ba8a04be28db6a72a67d5c239
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2
[11/18] Training with params: {'n_estimators': 10, 'max_depth': 5, 'min_samples_split': 2}


2025/11/30 13:52:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/30 13:52:10 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/30 13:52:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  → Accuracy: 1.0000, F1: 1.0000

🏃 View run Run_11 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/69c309f7ce6f457fae75f8041dab3124
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2
🏃 View run GridSearch_ParentRun at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2/runs/bbf829150ada48828e80398919bc9caa
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/2


RestException: INVALID_PARAMETER_VALUE: The run bbf829150ada48828e80398919bc9caa must be in the 'active' state. Current state is deleted.

# What else is there?  

In short: alot.  
What else would be possible:  
- write the entire train and test dataset to mlflow.
- write an evaluation set, where mlflow server itself compares the models using the eval set (maybe not enabled in compose)

The ways to waste time are nearly endless, and using every single feature will be a bit overkill. 